In [ ]:
import zipfile
import os

zip_file_path = '/kaggle/input/pnd-14-256x256/train_patches.zip'
target_folder = '/kaggle/working/train'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_folder)

print(f'Files extracted to {target_folder}')


In [ ]:
# Libraries imports

import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision.transforms import ToTensor
from fastai.vision.all import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.metrics import cohen_kappa_score, roc_auc_score, confusion_matrix
from PIL import Image
from fastai.metrics import Metric
import cv2
import matplotlib.pyplot as plt
import random

In [ ]:
# Constants
patch_size = 256  # Size of image patches
batch_size = 10  # Batch size
n_patches = 14  # Number of patches per image
TRAIN = '/kaggle/working/train'
LABELS = '/kaggle/input/trainlabels/train.csv'

mean = torch.tensor([0.803921, 0.596078, 0.729411])
std = torch.tensor([0.145098, 0.219607, 0.149019])

# Calculate alphas based on class distribution
class_counts = [2893, 2666, 1344, 1243, 1250, 1225]
total = sum(class_counts)
alphas = [total / count for count in class_counts]

# Normalize alphas
alphas_sum = sum(alphas)
normalized_alphas = [alpha / alphas_sum for alpha in alphas]
min_alpha = min(normalized_alphas)
scaled_alphas =  [alpha / min_alpha for alpha in normalized_alphas]

In [ ]:
# Data Preparation - Dataset definition
df = pd.read_csv(LABELS).set_index('image_id')
files = sorted(set([p[:32] for p in os.listdir(TRAIN)]))
df = df.loc[files].reset_index()

# Separate a portion for evaluation
rest_df, eval_df = train_test_split(df, test_size=0.1, stratify=df['isup_grade'], random_state=42)
rest_df = rest_df.reset_index(drop=True)
eval_df = eval_df.reset_index(drop=True)

# Train/Validation splitting
train_df, valid_df = train_test_split(rest_df, test_size=0.15, stratify=rest_df['isup_grade'], random_state=42)
train_df['split'] = 0
valid_df['split'] = 1
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)


df_final = pd.concat([train_df, valid_df]).reset_index(drop=True)

In [ ]:
# Pre-processing, Dataloading

# Get_x function to return image paths
def get_x(r):
    return [Path(TRAIN)/f'{r["image_id"]}_{i}.png' for i in range(n_patches)]

def get_y(r):
    return r['isup_grade']

def open_images_eval(fn):
    processed_imgs = []
    for f in fn:
        img = Image.open(f).convert('RGB')

        # Dynamic padding - Normalization
        padding = (0, 0, max(0, patch_size - img.width), max(0, patch_size - img.height))
        tfms = transforms.Compose([
            transforms.CenterCrop(patch_size),
            transforms.Pad(padding, fill=(255, 255, 255), padding_mode='constant'),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

        img_tensor = tfms(img)
        processed_imgs.append(img_tensor)

    return processed_imgs

def open_images(fn):
    processed_imgs = []
    for f in fn:
        img = Image.open(f).convert('RGB')

        # Apply dynamic padding and transformations/normalization
        tfms = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.CenterCrop(patch_size),
            transforms.Pad((0, 0, max(0, patch_size - img.width), max(0, patch_size - img.height)), fill=(255, 255, 255), padding_mode='constant'),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

        img_tensor = tfms(img)
        processed_imgs.append(img_tensor)

    return processed_imgs


def collate(batch):
    batch = [item for sublist in batch for item in sublist]  # Flatten the list of lists
    batch = torch.stack(batch)
    return batch

def custom_splitter():
    def _inner(_):
        train_idxs = df_final.index[df_final['split'] == 0].tolist()
        valid_idxs = df_final.index[df_final['split'] == 1].tolist()
        return train_idxs, valid_idxs
    return _inner


# DataLoaders (Training/Validation, Evaluation)
dblock = DataBlock(
    blocks=(TransformBlock(type_tfms=open_images), CategoryBlock),
    get_x=get_x,
    get_y=get_y,
    splitter=custom_splitter(),
    batch_tfms=[]
)

dls = dblock.dataloaders(df_final, bs=batch_size, collate_fn=collate)

dblock_eval = DataBlock(blocks=(TransformBlock(type_tfms=open_images_eval), CategoryBlock),
                        get_x=get_x,
                        get_y=get_y,
                        splitter=RandomSplitter(valid_pct=0))
dls_eval = dblock_eval.dataloaders(eval_df, bs=batch_size, collate_fn=collate)


In [ ]:
#Select a Random Image
random_index = random.randint(0, len(df_final)-1)
random_image_row = df_final.iloc[random_index]

#Load Image Patches
image_paths = get_x(random_image_row)  # Get paths for the patches
image_patches = open_images(image_paths)  # Load and preprocess patches

#Plot the Patches
fig, axes = plt.subplots(2, 7, figsize=(35, 10))  # Adjust the grid size
axes = axes.flatten()
for i, img_tensor in enumerate(image_patches):
    img = img_tensor.numpy().transpose((1, 2, 0))  # Convert tensor to numpy array and rearrange dimensions
    img = img * std.numpy() + mean.numpy()  # Un-normalize using mean and std
    img = np.clip(img, 0, 1)  # Clip values to be in the range [0, 1] for valid image display
    axes[i].imshow(img)
    axes[i].axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Model

# Mish activation function
class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_tensors[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x))
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)

# N patches aggregator (into one unified map)
class PatchAggregator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.aggregation = nn.Sequential(
            nn.Linear(in_features, 128),
            Mish(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        weights = self.aggregation(x)
        return (x * weights).sum(dim=1)

# Pooling : Adaptive Average and Max
class Pooling(nn.Module):
    def __init__(self):
        super().__init__()
        self.output_size = (1,1)
        self.ap = nn.AdaptiveAvgPool2d(self.output_size)
        self.mp = nn.AdaptiveMaxPool2d(self.output_size)
    def forward(self, x):
        return torch.cat([self.mp(x), self.ap(x)], 1)


# Model Architecture
class DenseNetModel(nn.Module):
    def __init__(self, n_classes=6):
        super().__init__()
        weights = DenseNet169_Weights.DEFAULT
        base_model = densenet169(weights=weights)
        layers = list(base_model.children())[:-1]
        feature_dim = list(base_model.children())[-1].in_features
        self.encoder = nn.Sequential(*layers)
        self.pooling = Pooling()
        self.aggregation = PatchAggregator(2 * feature_dim)
        self.head = nn.Sequential(
            nn.Linear(2 * feature_dim, 512),
            nn.BatchNorm1d(512),
            Mish(),
            nn.Dropout(0.5),
            nn.Linear(512, n_classes)
        )

    def forward(self, input):
        n_patches = len(input)
        batch_size, c, h, w = input[0].size()
        stacked_patches = torch.stack(input,1).view(-1,c,h,w)
        patch_features = self.encoder(stacked_patches)
        reduced_features = self.pooling(patch_features).view(batch_size, n_patches, -1)
        aggregated_features = self.aggregation(reduced_features)
        return self.head(aggregated_features)

In [ ]:
# Loss function and metrics definition

# Custom (weighted) Loss Function
class FocalLoss(nn.Module):
    def __init__(self, alphas, gamma, reduction):
        super().__init__()
        self.gamma = gamma
        self.reduction = reduction
        self.alphas = torch.tensor(alphas).float()

    def forward(self, inputs, targets):
        alphas = self.alphas.to(inputs.device)

        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        alpha = alphas[targets]
        F_loss = alpha * ((1 - pt) ** self.gamma) * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss


# Initialize the loss function with normalized alphas
focal_loss = FocalLoss(alphas=scaled_alphas, gamma=2, reduction='mean')

# Metric Multiclass RocAuc
class MulticlassRocAuc(Metric):
    def __init__(self):
        self.preds = []
        self.targets = []

    def reset(self):
        self.preds = []
        self.targets = []

    def accumulate(self, learn):
        preds, targs = learn.pred, learn.y
        self.preds.append(preds)
        self.targets.append(targs)

    @property
    def value(self):
        preds = torch.cat(self.preds)
        targets = torch.cat(self.targets)
        # Convert to one-hot format for multiclass ROC AUC calculation
        targets_one_hot = torch.nn.functional.one_hot(targets, num_classes=preds.size(-1))
        return roc_auc_score(targets_one_hot.cpu().numpy(), preds.cpu().numpy(), multi_class='ovr')

roc_auc_multiclass = MulticlassRocAuc()

In [ ]:
# Learner definition and learning rate

# Learner
learn = Learner(dls, DenseNetModel(), loss_func=focal_loss, opt_func=ranger, metrics=[roc_auc_multiclass, CohenKappa(weights='quadratic')])

learn.to_fp16()

# Gradient Clipping
learn.clip_grad = 1.0

# Find the optimal learning rate
learn.lr_find()

In [ ]:
# Learner
learn = Learner(dls, DenseNetModel(), loss_func=focal_loss, opt_func=ranger, metrics=[CohenKappa(weights='quadratic')])

learn.to_fp16()

# Gradient Clipping
learn.clip_grad = 1.0

In [ ]:
# Training and results [Over the validation set and on the evaluation set (unseen, unmasked images)]

# Training
learn.fit_one_cycle(30, 0.0002290867705596611, cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=24), SaveModelCallback(monitor='cohen_kappa_score', fname='best_model', with_opt=True)])

learn.load('best_model')

# Generate predictions on the evaluation dataset
preds, targs = learn.get_preds(dl=dls_eval.train)

# Convert predictions to class indices
pred_classes = torch.argmax(preds, dim=1)

# Calculate Kappa Score
kappa_score = cohen_kappa_score(targs.numpy(), pred_classes.numpy(), weights='quadratic')

# Calculate Confusion Matrix
conf_matrix = confusion_matrix(targs.numpy(), pred_classes.numpy())

# Convert softmax probabilities to class probabilities for each class
preds_probs = F.softmax(preds, dim=1)

# Convert targets to one-hot encoding to match the shape of preds_probs for ROC AUC calculation
targs_one_hot = F.one_hot(targs, num_classes=preds_probs.size(-1))

# Calculate ROC AUC score for multiclass classification
roc_auc = roc_auc_score(targs_one_hot.cpu().numpy(), preds_probs.cpu().numpy(), multi_class='ovr')

# Output the results
print(f"ROC AUC Score (Multiclass): {roc_auc}")
print(f"Kappa Score: {kappa_score}")
print("Confusion Matrix:")
print(conf_matrix)